## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-13-01-04-00 +0000,wsj,Answers to frequently asked questions about Th...,https://www.wsj.com/economy/economic-forecasti...
1,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...
2,2025-07-13-00-29-55 +0000,bbc,Trump says EU and Mexico face 30% tariff from ...,https://www.bbc.com/news/articles/cyvj13d9ylpo
3,2025-07-12-23-56-51 +0000,nypost,Trump defends Pam Bondi hours after FBI boss K...,https://nypost.com/2025/07/12/us-news/trump-de...
4,2025-07-12-23-48-24 +0000,bbc,Trump's tariffs are looming large over the UK’...,https://www.bbc.com/news/articles/c5yp5qzeer6o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2317/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
30,trump,16
100,attempt,4
99,assassination,3
32,mexico,3
51,tariffs,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
9,2025-07-12-20-56-38 +0000,nypost,Secret Service officials were aware of ‘classi...,https://nypost.com/2025/07/12/us-news/top-secr...,40
15,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...,38
18,2025-07-12-16-17-04 +0000,nypost,Widow of killed fire chief not satisfied with ...,https://nypost.com/2025/07/12/us-news/corey-co...,35
3,2025-07-12-23-56-51 +0000,nypost,Trump defends Pam Bondi hours after FBI boss K...,https://nypost.com/2025/07/12/us-news/trump-de...,33
30,2025-07-12-02-08-29 +0000,nypost,Trump grants ICE ability to arrest ‘SLIMEBALL’...,https://nypost.com/2025/07/11/us-news/trump-gr...,31


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
9,40,2025-07-12-20-56-38 +0000,nypost,Secret Service officials were aware of ‘classi...,https://nypost.com/2025/07/12/us-news/top-secr...
1,23,2025-07-13-01-00-00 +0000,wsj,Economists expect stronger growth and job crea...,https://www.wsj.com/economy/economists-see-low...
15,22,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...
3,17,2025-07-12-23-56-51 +0000,nypost,Trump defends Pam Bondi hours after FBI boss K...,https://nypost.com/2025/07/12/us-news/trump-de...
32,16,2025-07-12-01-53-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...
24,14,2025-07-12-09-30-00 +0000,wsj,Pittsburgh is striving to turn its old steel m...,https://www.wsj.com/tech/ai/can-pittsburghs-ol...
21,14,2025-07-12-14-00-00 +0000,wsj,The White House has an explanation for the nea...,https://www.wsj.com/politics/policy/trump-new-...
5,13,2025-07-12-23-45-33 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...
29,13,2025-07-12-02-56-04 +0000,nypost,LA Mayor Karen Bass signs order to provide cas...,https://nypost.com/2025/07/11/us-news/la-mayor...
8,12,2025-07-12-22-29-02 +0000,nypost,Trump set to join prestigious presidential roa...,https://nypost.com/2025/07/12/us-news/trump-se...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
